### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)


# Import to run %sh magic cell

In [31]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.


In [32]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

### Displaying shell command output:
**Note**: Will not display _asadm_ or _aql_ executables output.

In [33]:
String command = "pwd";
 
try {
    Process process = Runtime.getRuntime().exec(command);
 
    BufferedReader reader = new BufferedReader(
            new InputStreamReader(process.getInputStream()));
    String line;
    while ((line = reader.readLine()) != null) {
        System.out.println(line);
    }
 
    reader.close();
 
} catch (IOException e) {
    e.printStackTrace();
}

/home/training/student-workbook/v7.0/jn/codeExamples


# Add Java Client POM Dependency

In [34]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [35]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Accessing a record on the Aerospike Server
We build the Key object in Java
We need the namespace and the record digest to find the record on the server.
The record digest is computed by the client library using application provided key (integer, string or byte-array) and the set name. If not in a set, use null for set name.

<img src="./graphics/RecordKey.png"
     alt="Record Digest"
     style="float: left; margin-right: 10px;"
     width="600"
     height="400"/>

# Insert test records
* Our one node server is already running on localhost (same server as the Jupyter Notebook server).
* Unlike in iPython kernel for Jupyter Notebook, we can only execute a single line of shell command in IJava kernel.

In [36]:
%sh aql -c "set key_send true"

In [37]:
%sh aql -f "./aqlScripts/insert.aql"

In [38]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
Read back the record.
Record values are:
(gen:1),(exp:0),(bins:(name:Jack),(age:26))


In [39]:
%sh asadm --enable -e "manage sindex create numeric idx_age ns test set testset bin age"

In [40]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Exp.Type;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,30));
QueryPolicy qp = new QueryPolicy();

Exp exp = Exp.val(false);  //init to false
// Add Expression
/*
qp.filterExp = Exp.build(Exp.or( 
  Exp.eq(Exp.key(Type.STRING), Exp.val("key1")),   //Jack
  Exp.eq(Exp.key(Type.STRING), Exp.val("key2")),   //Jill
  Exp.eq(Exp.key(Type.STRING), Exp.val("key8"))    //Sam, age 12, SI age reject.
  )
  );

*/
String[] keys = {"key1", "key2", "key8"};
for (int i=0; i<keys.length; i++)
{
   exp = Exp.or(exp, Exp.eq(Exp.key(Type.STRING), Exp.val(keys[i]))); 
}
qp.filterExp = Exp.build(exp);

RecordSet rs = client.query(qp, stmt);
while (rs.next()){
  Record r = rs.getRecord();
  System.out.println(r);
}

(gen:1),(exp:0),(bins:(name:Jill),(age:20))
(gen:1),(exp:0),(bins:(name:Jack),(age:26))


In [41]:
//CLASS Cell
class myTest {
  public void foo (Record rec) {
    System.out.println("Calling myTest:foo() to print rec:"+rec);
  }
}

In [42]:
myTest myobj = new myTest();
myobj.foo(record);

Calling myTest:foo() to print rec:(gen:1),(exp:0),(bins:(name:Jack),(age:26))


# Cleanup

In [43]:
%sh asadm --enable -e "manage sindex delete idx_age ns test set testset"

In [44]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"